In [1]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
import random
from scipy import stats
import random
import pywt.data
from PIL import Image
import pandas as pd
import seaborn as sns
import os
import pickle

In [2]:
# assign directory
ROOT_DIR = '/Users/brandonmarks/Desktop/Research Materials/hierarchical-bayesian-model-validation/'
data_dir = ROOT_DIR + 'raw-data/agriVision/toy-agriVision-RGB-cleaned'
 
# iterate over files in that directory
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)

In [3]:
image = Image.open(file_list[0]).convert('L')
first_image = pywt.wavedec2(image, 'db1')
layer_len = len(first_image)



In [4]:
#for k in range(len(file_list)):
layer_arr = [0] * len(file_list) * (layer_len - 1) * 3 
orientation = [0] * len(file_list) * (layer_len - 1) * 3 
data_arr = [0] * len(file_list) * (layer_len - 1) * 3 
c = 0
for k in range(len(file_list)):
    image = Image.open(file_list[k]).convert('L')
    image = np.array(image)
   
    transformed = pywt.wavedec2(image, 'db1')
    direction_names = ['H', 'V', 'D']

    for i in range(1, layer_len): 
        for j in range(len(transformed[i])):
            
            arr = np.array(transformed[i][j])
            layer_arr[c] = i+1
            orientation[c] =  direction_names[j]
            data_arr[c] = arr.flatten()
            c += 1

In [5]:
df = pd.DataFrame()

df["Layer"] = layer_arr
df["Orientation"] = orientation
df["Data"] = data_arr
new_df = pd.DataFrame(columns=["Channel","Layer", "Orientation", "Data"])
for lo, sf in df.groupby(["Layer", "Orientation"])[["Data"]]:#.agg(lambda sf: np.concatenate(sf["Data"].tonumpy()))
    new_df.loc[len(new_df.index)] = ["Red", lo[0], lo[1],  np.hstack(sf['Data'])]
new_df

,Channel,Layer,Orientation,Data
0,Red,2,D,"[-54.38085937500006, 60.472656250000114, 4.378..."
1,Red,2,H,"[63.78320312500007, 51.61328125000003, 8.80468..."
2,Red,2,V,"[-89.6035156250001, -4.45703125, -11.328124999..."
3,Red,3,D,"[8.62500000000001, 6.179687500000009, -2.51953..."
4,Red,3,H,"[-13.429687500000018, -9.156250000000009, 9.32..."
5,Red,3,V,"[-29.87500000000003, -33.98437500000004, -21.8..."
6,Red,4,D,"[0.28906250000000033, -3.7578125000000018, -2...."
7,Red,4,H,"[-0.08593750000000022, -7.117187500000009, -27..."
8,Red,4,V,"[-1.6484375000000018, -6.757812500000007, -15...."
9,Red,5,D,"[-0.2031250000000001, 1.281250000000001, 1.859..."


In [6]:
def convert_to_wavelet_basis(folder_dir, color, basis="db1"):
    file_list = [os.path.join(folder_dir, filename) for filename in os.listdir(folder_dir)]
    #Setup df Dict
    image = Image.open(file_list[0]).convert('L')
    first_image = pywt.wavedec2(image, basis)
    layer_len = len(first_image)
    print(str(layer_len) + " layers being used")
    
    color_dict = {"Red":0, "Green":1, "Blue":2, "Gray":3, "Infrared": 4}
    c = color_dict[color]

    #Fill DF DICT
    layer_arr = [0] * len(file_list) * (layer_len - 1) * 3 
    orientation = [0] * len(file_list) * (layer_len - 1) * 3 
    data_arr = [0] * len(file_list) * (layer_len - 1) * 3 
    cnt = 0
    for k in range(len(file_list)):
        if c >= 3:
            image = np.array(Image.open(file_list[k]).convert('L'))
        else:
            image = np.array(Image.open(file_list[k]))[:,:,c]
    
        transformed = pywt.wavedec2(image, 'db1')
        direction_names = ['H', 'V', 'D']

        for i in range(1, layer_len): 
            for j in range(len(transformed[i])):
                
                arr = np.array(transformed[i][j])
                layer_arr[cnt] = i+1
                orientation[cnt] =  direction_names[j]
                data_arr[cnt] = arr.flatten()
                cnt += 1

    df = pd.DataFrame()

    df["Layer"] = layer_arr
    df["Orientation"] = orientation
    df["Data"] = data_arr
    new_df = pd.DataFrame(columns=["Channel","Layer", "Orientation", "Data"])
    for lo, sf in df.groupby(["Layer", "Orientation"])[["Data"]]:#.agg(lambda sf: np.concatenate(sf["Data"].tonumpy()))
        new_df.loc[len(new_df.index)] = [color, lo[0], lo[1],  np.hstack(sf['Data'])]
    
    return new_df

In [7]:
red_converted = convert_to_wavelet_basis(data_dir, "Red")
red_converted

10 layers being used


,Channel,Layer,Orientation,Data
0,Red,2,D,"[-53.76953125000005, 60.29492187500012, -0.917..."
1,Red,2,H,"[63.179687500000064, 50.95117187499999, 6.7343..."
2,Red,2,V,"[-87.5273437500001, -9.361328125, -21.09375000..."
3,Red,3,D,"[7.285156250000005, 7.214843750000008, -2.5312..."
4,Red,3,H,"[-12.089843750000012, -8.589843750000009, 8.83..."
5,Red,3,V,"[-28.53515625000003, -34.38671875000006, -21.0..."
6,Red,4,D,"[0.28906250000000033, -3.7578125000000018, -0...."
7,Red,4,H,"[-0.08593750000000022, -7.117187500000009, -28..."
8,Red,4,V,"[-1.6484375000000018, -6.757812500000007, -14...."
9,Red,5,D,"[-0.2031250000000001, 1.281250000000001, 1.859..."


In [8]:
green_converted = convert_to_wavelet_basis(data_dir, "Green")
green_converted

10 layers being used


,Channel,Layer,Orientation,Data
0,Green,2,D,"[-54.748046875000064, 60.771484375000185, 6.72..."
1,Green,2,H,"[64.33007812500007, 52.45507812500003, 16.5664..."
2,Green,2,V,"[-90.8027343750001, 0.33007812499994316, -6.49..."
3,Green,3,D,"[9.527343750000007, 5.507812500000005, -2.4296..."
4,Green,3,H,"[-14.332031250000014, -9.648437500000002, 9.42..."
5,Green,3,V,"[-30.77734375000003, -34.00781250000004, -22.3..."
6,Green,4,D,"[0.28906250000000033, -3.7578125000000018, -2...."
7,Green,4,H,"[-0.08593750000000022, -7.117187500000009, -27..."
8,Green,4,V,"[-1.6484375000000018, -6.757812500000007, -15...."
9,Green,5,D,"[-0.2031250000000001, 1.281250000000001, 1.859..."


In [9]:
blue_converted = convert_to_wavelet_basis(data_dir, "Blue")
blue_converted

10 layers being used


,Channel,Layer,Orientation,Data
0,Blue,2,D,"[-79.18359375000007, 59.93750000000008, -0.888..."
1,Blue,2,H,"[90.8710937500001, 49.05078124999999, -14.1269..."
2,Blue,2,V,"[-121.25781250000013, -5.492187500000057, -8.1..."
3,Blue,3,D,"[8.69140625000001, 37.976562500000036, -2.8476..."
4,Blue,3,H,"[-13.496093750000018, -23.164062500000018, 9.3..."
5,Blue,3,V,"[-31.941406250000036, -57.84375000000006, -21...."
6,Blue,4,D,"[0.28906250000000033, -5.757812500000003, -8.9..."
7,Blue,4,H,"[-0.08593750000000022, -5.117187500000008, -25..."
8,Blue,4,V,"[-1.6484375000000018, -8.75781250000001, -30.5..."
9,Blue,5,D,"[-0.2031250000000001, 1.281250000000001, 1.859..."


In [10]:
gray_converted = convert_to_wavelet_basis(data_dir, "Gray")
gray_converted

10 layers being used


,Channel,Layer,Orientation,Data
0,Gray,2,D,"[-54.38085937500006, 60.472656250000114, 4.378..."
1,Gray,2,H,"[63.78320312500007, 51.61328125000003, 8.80468..."
2,Gray,2,V,"[-89.6035156250001, -4.45703125, -11.328124999..."
3,Gray,3,D,"[8.62500000000001, 6.179687500000009, -2.51953..."
4,Gray,3,H,"[-13.429687500000018, -9.156250000000009, 9.32..."
5,Gray,3,V,"[-29.87500000000003, -33.98437500000004, -21.8..."
6,Gray,4,D,"[0.28906250000000033, -3.7578125000000018, -2...."
7,Gray,4,H,"[-0.08593750000000022, -7.117187500000009, -27..."
8,Gray,4,V,"[-1.6484375000000018, -6.757812500000007, -15...."
9,Gray,5,D,"[-0.2031250000000001, 1.281250000000001, 1.859..."


In [11]:
infra_converted = convert_to_wavelet_basis(data_dir, "Infrared")
infra_converted

10 layers being used


,Channel,Layer,Orientation,Data
0,Infrared,2,D,"[-54.38085937500006, 60.472656250000114, 4.378..."
1,Infrared,2,H,"[63.78320312500007, 51.61328125000003, 8.80468..."
2,Infrared,2,V,"[-89.6035156250001, -4.45703125, -11.328124999..."
3,Infrared,3,D,"[8.62500000000001, 6.179687500000009, -2.51953..."
4,Infrared,3,H,"[-13.429687500000018, -9.156250000000009, 9.32..."
5,Infrared,3,V,"[-29.87500000000003, -33.98437500000004, -21.8..."
6,Infrared,4,D,"[0.28906250000000033, -3.7578125000000018, -2...."
7,Infrared,4,H,"[-0.08593750000000022, -7.117187500000009, -27..."
8,Infrared,4,V,"[-1.6484375000000018, -6.757812500000007, -15...."
9,Infrared,5,D,"[-0.2031250000000001, 1.281250000000001, 1.859..."
